# Data

## Import libraries and Set Options

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, ParameterGrid
from sklearn.preprocessing import (StandardScaler, OneHotEncoder,
                                   PolynomialFeatures)
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics
from sklearn.metrics import mean_absolute_percentage_error

from daftpy.daftmodel import split_data, scores_statistics, metrics_regression, plot_learning_curves, compare_models

import joblib

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')

## Load Data

In [3]:
sale_data = pd.read_csv('data_available/sale_data_post_out.csv', sep=',')
sale_data.shape  #### cambiar esto

(7387, 34)

In [4]:
features = [
    
    'price',
    'floor_area',
    'views',
    'latitude',
    'longitude',
#    'bedroom',
    'bathroom',
#    'sale_type',
    'type_house',
#    'postcode',
#    'state_district',
#    'county',
#    'city_district',
#    'road',
#    'place',
    'code',
#    'admin1',
#    'cities'
]

data = sale_data[features].copy()
data.shape

(7387, 8)

In [5]:
num_features = list(data.select_dtypes('number').columns) #X_train
num_features.remove('price')
cat_features =  list(data.select_dtypes('object').columns)

num_feat_df = pd.DataFrame({'numerical': num_features}) 
cat_feat_df = pd.DataFrame({'categorical': cat_features})

feat_df = num_feat_df.merge(cat_feat_df, how='left', left_index=True, right_index=True).fillna(' ')
feat_df

,numerical,categorical
0,floor_area,type_house
1,views,code
2,latitude,
3,longitude,
4,bathroom,


In [6]:
levels_type_house = data.type_house.unique()
levels_code = data.code.unique()

cat_feat_df['n_levels'] = [data.type_house.nunique(), data.code.nunique()]
cat_feat_df['sample'] = [levels_type_house, levels_code[:5]]
cat_feat_df # mejorar

,categorical,n_levels,sample
0,type_house,2,"[house, apartment]"
1,code,137,"[D09, T23, F91, V95, R32]"


## Split Data

In [7]:
X_train, X_test, y_train, y_test = split_data(data=data, 
                                              target='price', 
                                              test_size=.2, 
                                              output='X_y_train_test',
                                              random_state=42)

X_train: (5909, 7) 
X_test: (1478, 7) 
y_train: (5909,) 
y_test: (1478,) 



------------

# Metrics

In [8]:
scoring = {'r2': 'r2', 
           'MAE': 'neg_mean_absolute_error', 
           'MAPE': 'neg_mean_absolute_percentage_error', 
           'RMSE': 'neg_root_mean_squared_error'}

# Models

## Multiple Linear Regression

En estadística, el **coeficiente de determinación**, denominado R² y pronunciado R cuadrado, es un estadístico usado en el contexto de un modelo estadístico cuyo principal propósito es predecir futuros resultados o probar una hipótesis. El coeficiente determina la calidad del modelo para replicar los resultados, y la proporción de variación de los resultados que puede explicarse por el modelo.

Es el porcentaje de la variación en la variable de respuesta que es explicado por un modelo lineal. Es decir:

R-cuadrado = Variación explicada / variación total

El R-cuadrado siempre está entre 0 y 100%:

https://scikit-learn.org/stable/modules/cross_validation.html

In [9]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
#    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  # No hace nada si ya transformadas
                                      #handle_unknown='ignore'
    ])
# Las transforme antes para evitar problemas no las variables a la hora de predecir e el test_set...

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


lr_pipe_estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', # esto lo puedo agnadir en los otros pipes
                              fill_value=None)),
    ('regressor', LinearRegression())
    ])

In [10]:
scores_lr, scores_resume_lr = scores_statistics(estimator=lr_pipe_estimator, 
                                                scoring_dict=scoring, 
                                                X_train=X_train, 
                                                y_train=y_train, 
                                                cv=10, 
                                                return_train_score=False)

r2 mean: 0.6581505970522035
r2 std: 0.03247492814306588 

MAE mean: -108719.58221709491
MAE std: 5568.4947907050655 

MAPE mean: -0.32069981130761466
MAPE std: 0.015889144852600887 

RMSE mean: -176983.98634043717
RMSE std: 12945.409397794008 



### Overfitting-Underfitting Analysis

#### Evaluating on the Training and Test Sets

We can identify if a machine learning model has overfit by first evaluating the model on the training dataset and then evaluating the same model on a holdout test dataset.

If the performance of the model on the training dataset is significantly better than the performance on the test dataset, then the model may have overfit the training dataset.

In [11]:
lr = lr_pipe_estimator.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = lr.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = lr.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.6739892726197527
MAE: 106081.66588347829
MAPE: 0.31183387356867376
RMSE: 173716.41014659725

Performance on the test set:
 ----------
R²: 0.6468123069342815
MAE: 111099.76275851484
MAPE: 0.321239513969634
RMSE: 183180.30027983472



#### Learning Curves

The model is underfitting the training data, so we need to use more complex models or come up with better features.

-------------

------------

In [12]:
#joblib.dump(lr, 'models/linear_regression_01-11-2021.plk')

-------------------

## Polynomial Regression

In [13]:
num_pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('std_scaler', StandardScaler()),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


poly_pipe_estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', LinearRegression())
                           ])

In [14]:
#pd.DataFrame(grid_search.cv_results_) # plotearlo

In [15]:
scores = scores_statistics(estimator=poly_pipe_estimator, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

r2 mean: 0.7509266721418995
r2 std: 0.029110140197970813 

MAE mean: -94476.7902938415
MAE std: 3679.6981417813968 

MAPE mean: -0.2738010265729388
MAPE std: 0.010822070061438843 

RMSE mean: -150821.55756749702
RMSE std: 10517.670372640698 



In [16]:
poly = poly_pipe_estimator.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = poly.predict(X_train)
metrics_regression(y_test=y_train, 
                    y_pred=y_pred_train, 
                    squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = poly.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)


Performance on the training set:
 ----------
R²: 0.7737738943989698
MAE: 90784.73634486872
MAPE: 0.2629339442693284
RMSE: 144709.23924104206

Performance on the test set:
 ----------
R²: 0.7341027244460809
MAE: 97188.98815136666
MAPE: 0.271332077376683
RMSE: 158939.89776053437



## Support Vector Machines

## K Nearest Neighbors Regressor

In [17]:
from sklearn.neighbors import KNeighborsRegressor

In [18]:
num_pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ('std_scaler', StandardScaler()),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


knnr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', KNeighborsRegressor())
                           ])

In [19]:
knnr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', KNeighborsRegressor(n_neighbors=10,
                                      
                                     ))
                           ])

scores = scores_statistics(estimator=knnr_pipe_est, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=5, 
                           return_train_score=False)

r2 mean: 0.71811523350896
r2 std: 0.047035588574515515 

MAE mean: -94951.2568978797
MAE std: 3616.1771208504674 

MAPE mean: -0.2673478345689654
MAPE std: 0.006640880486903855 

RMSE mean: -160776.974205796
RMSE std: 12246.337737638607 



In [20]:
knnr = knnr_pipe_est.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = knnr.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = knnr.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.7808931429920254
MAE: 83952.01976645795
MAPE: 0.23402768381886224
RMSE: 142414.06577198117

Performance on the test set:
 ----------
R²: 0.7149744015727351
MAE: 94532.48254397836
MAPE: 0.25724751266987317
RMSE: 164557.59012118593



## Decission Tree Regressor

In [21]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


dtr_pipe_estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', DecisionTreeRegressor()) # max_depth=9spoiler
                           ])

In [22]:
#pd.DataFrame(grid_search.cv_results_)

In [57]:
dtr_pipe_estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', DecisionTreeRegressor(max_depth=10,
                                        min_samples_leaf=16,
                                        min_samples_split=43,
                                        max_leaf_nodes=68
                                       )) 
                           ])

scores = scores_statistics(estimator=dtr_pipe_estimator, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=5, 
                           return_train_score=False)

r2 mean: 0.7347439806116778
r2 std: 0.024814399630088618 

MAE mean: -95341.41736197876
MAE std: 1686.2968105662758 

MAPE mean: -0.27692435189721787
MAPE std: 0.006257078986299098 

RMSE mean: -156514.0008343089
RMSE std: 7832.77912950293 



In [24]:
dtr = dtr_pipe_estimator.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = dtr.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = dtr.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.7927341211787164
MAE: 86542.06835968455
MAPE: 0.25752620175637614
RMSE: 138512.4486203107

Performance on the test set:
 ----------
R²: 0.7304601782182829
MAE: 96912.11046384604
MAPE: 0.27164761682651445
RMSE: 160024.8593006069



## Voting Regressor

In [29]:
from sklearn.ensemble import VotingRegressor

In [30]:
scoring_comp = {'r2': 'r2',
                'MAPE': 'neg_mean_absolute_percentage_error'}

Con el random forest mejora un poco pero el rfr sigue siendo mejor

In [40]:
models_r2 = {'poly': 73.41, 'knn': 71.49, 'dt': 73.04}

tot = 0
for key in models_r2:
    tot += models_r2[key]

models_weigth = {}
models_weigth_list = []
for key in models_r2:
    weight = models_r2[key] / tot
    models_weigth[key] = weight
    models_weigth_list.append(weight)
models_weigth
#models_weigth_list

{'poly': 0.33683582637423143,
 'knn': 0.3280260622189593,
 'dt': 0.33513811140680927}

In [41]:
polyr = poly_pipe_estimator
knnr = knnr_pipe_est
dtr = dtr_pipe_estimator
#rfr = rfr_pipe_est

voting_reg = VotingRegressor(
    estimators= [('poly', polyr),('knn', knnr),('dt', dtr)], #('rfr',rfr)
    weights= models_weigth_list,
)

models_dict = {'Polynomial Regression': polyr, 
               'K Nearest Neighbors Regressor': knnr, 
               'Decission Tree Regressor': dtr, 
             #  'Random Forest Regressor': rfr,
               'Voting Regressor': voting_reg}

for key in models_dict:
    print(key)  
    scores = compare_models(estimator=models_dict[key], 
                            scoring_dict=scoring_comp, 
                            X_train=X_train, 
                            y_train=y_train, 
                            cv=10, 
                            return_train_score=False)

Polynomial Regression
r2 mean: 0.7509266721418995
MAPE mean: -0.2738010265729388
----------
K Nearest Neighbors Regressor
r2 mean: 0.723478756178525
MAPE mean: -0.263912331770578
----------
Decission Tree Regressor
r2 mean: 0.7193170788329176
MAPE mean: -0.28016911561435026
----------
Voting Regressor
r2 mean: 0.7833601411843516
MAPE mean: -0.2349531052600688
----------


In [42]:
voting_reg.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = voting_reg.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = voting_reg.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.8250917470379119
MAE: 76384.77001301781
MAPE: 0.2170076639656279
RMSE: 127241.87252301836

Performance on the test set:
 ----------
R²: 0.77813329415568
MAE: 83715.56093234899
MAPE: 0.22537719887227378
RMSE: 145185.10172562115



## Bagging with Polynomial Regression

In [17]:
from sklearn.ensemble import BaggingRegressor

In [62]:
bag_regression = BaggingRegressor(
    
    base_estimator=LinearRegression(),
    n_estimators=500, 
    max_samples=3000, 
#    max_features=50, 
#    bootstrap=True, 
#    bootstrap_features=False, 
#    oob_score=True, 
#    warm_start=False, 
    n_jobs=-1, 
    random_state=42, 
#    verbose=0
)

'''
    base_estimator=LinearRegression(),
    n_estimators=500, 
    max_samples=2000, 
#    max_features=1.0, 
#    bootstrap=True, 
#    bootstrap_features=False, 
#    oob_score=True, 
#    warm_start=False, 
    n_jobs=-1, 
    random_state=42, 
#    verbose=0
----------------
R²: 0.7724293692138182
R²: 0.7353834187513661
'''

'\n    base_estimator=LinearRegression(),\n    n_estimators=500, \n    max_samples=2000, \n#    max_features=1.0, \n#    bootstrap=True, \n#    bootstrap_features=False, \n#    oob_score=True, \n#    warm_start=False, \n    n_jobs=-1, \n    random_state=42, \n#    verbose=0\n----------------\nR²: 0.7724293692138182\nR²: 0.7353834187513661\n'

In [63]:
num_pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('std_scaler', StandardScaler()),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


br_pipe_estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', bag_regression)
                           ])

In [64]:
scores = scores_statistics(estimator=br_pipe_estimator, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=5, 
                           return_train_score=False)

r2 mean: 0.7484789696386661
r2 std: 0.017753527029554746 

MAE mean: -95150.15336594477
MAE std: 1727.5519094446418 

MAPE mean: -0.2763390928317644
MAPE std: 0.007261276392903534 

RMSE mean: -152427.59771405012
RMSE std: 4919.43221337097 



In [65]:
bag_reg = br_pipe_estimator.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = bag_reg.predict(X_train)
metrics_regression(y_test=y_train, 
                    y_pred=y_pred_train, 
                    squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = bag_reg.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.773131274043742
MAE: 90769.90056477503
MAPE: 0.2623421235946354
RMSE: 144914.62481491736

Performance on the test set:
 ----------
R²: 0.735524880514952
MAE: 96883.90805520506
MAPE: 0.26972445640914205
RMSE: 158514.28152992387



## Random Forest Regressor

In [71]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


rfr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', RandomForestRegressor(n_estimators=100, 
                               #         *, 
                                        criterion='squared_error',
                                        max_depth=None, 
                                        min_samples_split=2, 
                                        min_samples_leaf=1, 
                                        min_weight_fraction_leaf=0.0, 
                                        max_features='auto', 
                                        max_leaf_nodes=None, 
                                        min_impurity_decrease=0.0, 
                                        bootstrap=True, 
                                        oob_score=False, 
                                        n_jobs=None, 
                                        random_state=None, 
                                        verbose=0, 
                                        warm_start=False, 
                                        ccp_alpha=0.0, 
                                        max_samples=None)
    )
                           ])

In [91]:
param_grid = [
    {
#        'preprocessor__num__poly__degree': [1, 3],
        'regressor__n_estimators': [310,315,320,325,330], 
 #       'regressor__max_depth': range(2, 15),#[10],
  #      'regressor__min_samples_split':range(30, 50), #[43],
   #     'regressor__min_samples_leaf': range(5, 20),#[16],
    #    'regressor__max_leaf_nodes': range(40, 80),#[68],
     #   'regressor__n_jobs': [-1],
    },
    #{'regressor__n_estimators': range(41,70)}, #[3,10,30,40,50,60]
]


grid_search = GridSearchCV(estimator=rfr_pipe_est, 
                           param_grid=param_grid,
                           cv=5, 
                           n_jobs=-1,
                        #   scoring='neg_mean_squared_error',
                        #   return_train_score=True
                          )

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'regressor__n_estimators': 320}
0.8078519445963483


In [76]:
#pd.DataFrame(grid_search.cv_results_).tail()

In [92]:
rfr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', RandomForestRegressor(
        n_estimators=320, #325
#        max_depth=10,
#        min_samples_split=43,
#        min_samples_leaf=16,
#        max_leaf_nodes=68,
        n_jobs=-1,
                                       )) 
                           ])
#estimator

scores = scores_statistics(estimator=rfr_pipe_est, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

r2 mean: 0.8104012260704889
r2 std: 0.03305791905049029 

MAE mean: -76229.51641800112
MAE std: 3091.553179636989 

MAPE mean: -0.21519392947248192
MAPE std: 0.010073158736609784 

RMSE mean: -130985.59535647393
RMSE std: 9601.401676860758 



In [93]:
rfr = rfr_pipe_est.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = rfr.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = rfr.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.974280292530381
MAE: 28019.635152415805
MAPE: 0.07872468600398314
RMSE: 48793.04402528927

Performance on the test set:
 ----------
R²: 0.8077219486102967
MAE: 77397.63373012516
MAPE: 0.21008810092531388
RMSE: 135157.71524023675



The model is overfitting the training data. Necesitamos regularizar!!

## Extremely Randomized Trees

In [96]:
from sklearn.ensemble import ExtraTreesRegressor

In [97]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


etr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', ExtraTreesRegressor(n_estimators=100, 
                            #          *, 
                                      criterion='squared_error', 
                                      max_depth=None, 
                                      min_samples_split=2, 
                                      min_samples_leaf=1, 
                                      min_weight_fraction_leaf=0.0, 
                                      max_features='auto', 
                                      max_leaf_nodes=None, 
                                      min_impurity_decrease=0.0, 
                                      bootstrap=False, 
                                      oob_score=False, 
                                      n_jobs=-1, 
                                      random_state=None, 
                                      verbose=0, 
                                      warm_start=False, 
                                      ccp_alpha=0.0, 
                                      max_samples=None)
    )
                           ])

In [109]:
param_grid = [
    {
#        'preprocessor__num__poly__degree': [1, 3],
        'regressor__n_estimators': [95,100,105,110,115], 
 #       'regressor__max_depth': range(2, 15),#[10],
  #      'regressor__min_samples_split':range(30, 50), #[43],
   #     'regressor__min_samples_leaf': range(5, 20),#[16],
    #    'regressor__max_leaf_nodes': range(40, 80),#[68],
     #   'regressor__n_jobs': [-1],
    },
    #{'regressor__n_estimators': range(41,70)}, #[3,10,30,40,50,60]
]


grid_search = GridSearchCV(estimator=etr_pipe_est, 
                           param_grid=param_grid,
                           cv=5, 
                           n_jobs=-1,
                        #   scoring='neg_mean_squared_error',
                        #   return_train_score=True
                          )

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'regressor__n_estimators': 100}
0.8133204632387004


In [110]:
etr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', ExtraTreesRegressor(n_estimators=100, 
                            #          *, 
                                      criterion='squared_error', 
                                      max_depth=None, 
                                      min_samples_split=2, 
                                      min_samples_leaf=1, 
                                      min_weight_fraction_leaf=0.0, 
                                      max_features='auto', 
                                      max_leaf_nodes=None, 
                                      min_impurity_decrease=0.0, 
                                      bootstrap=False, 
                                      oob_score=False, 
                                      n_jobs=-1, 
                                      random_state=None, 
                                      verbose=0, 
                                      warm_start=False, 
                                      ccp_alpha=0.0, 
                                      max_samples=None)
    )
                           ])

scores = scores_statistics(estimator=etr_pipe_est, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

r2 mean: 0.8151091606774411
r2 std: 0.03406342439589219 

MAE mean: -76142.55296024836
MAE std: 3838.1940696224056 

MAPE mean: -0.21517666470873778
MAPE std: 0.010193661171624362 

RMSE mean: -129293.73608983801
RMSE std: 9606.194218053794 



In [111]:
etr = etr_pipe_est.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = etr.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = etr.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.9999998766477945
MAE: 4.400067693349128
MAPE: 1.5787791393412072e-05
RMSE: 106.85586396750058

Performance on the test set:
 ----------
R²: 0.8192602046733184
MAE: 76957.01573071719
MAPE: 0.20494747422113435
RMSE: 131039.69653046742



## AdaBoost Regressor

In [112]:
from sklearn.ensemble import AdaBoostRegressor

In [125]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


adaboost_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', AdaBoostRegressor(base_estimator=DecisionTreeRegressor(), 
                              #      *, 
                                    n_estimators=50, 
                                    learning_rate=1.0, 
                                    loss='linear', 
                                    random_state=42))
                           ])

In [134]:
param_grid = [
    {
#        'preprocessor__num__poly__degree': [1, 3],
        'regressor__n_estimators': [2,3,4,5,6,7,8,9,10,50], 
 #       'regressor__max_depth': range(2, 15),#[10],
  #      'regressor__min_samples_split':range(30, 50), #[43],
   #     'regressor__min_samples_leaf': range(5, 20),#[16],
    #    'regressor__max_leaf_nodes': range(40, 80),#[68],
     #   'regressor__n_jobs': [-1],
    },
    #{'regressor__n_estimators': range(41,70)}, #[3,10,30,40,50,60]
]


grid_search = GridSearchCV(estimator=adaboost_pipe_est, 
                           param_grid=param_grid,
                           cv=5, 
                           n_jobs=-1,
                        #   scoring='neg_mean_squared_error',
                        #   return_train_score=True
                          )

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'regressor__n_estimators': 3}
0.6504976212690231


In [135]:
adaboost_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', AdaBoostRegressor(base_estimator=None, 
                              #      *, 
                                    n_estimators=3, 
                                    learning_rate=1.0, 
                                    loss='linear', 
                                    random_state=42))
                           ])

scores = scores_statistics(estimator=adaboost_pipe_est, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

r2 mean: 0.6422642094394672
r2 std: 0.045081241398125756 

MAE mean: -122834.34681543028
MAE std: 4608.233441167535 

MAPE mean: -0.4030357828910572
MAPE std: 0.022817423528780835 

RMSE mean: -180430.7810183822
RMSE std: 8430.422906990498 



In [136]:
adaboost = adaboost_pipe_est.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = adaboost.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = adaboost.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.6825632436915089
MAE: 116749.91257832848
MAPE: 0.38086430496936213
RMSE: 171416.84878404267

Performance on the test set:
 ----------
R²: 0.6400955083561792
MAE: 122317.9107731734
MAPE: 0.37215236374014693
RMSE: 184913.92579798016



In [139]:
from sklearn.ensemble import GradientBoostingRegressor

In [125]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'


gb_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', GradientBoostingRegressor())
                           ])

In [135]:
gb_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', AdaBoostRegressor(base_estimator=None, 
                              #      *, 
                                    n_estimators=3, 
                                    learning_rate=1.0, 
                                    loss='linear', 
                                    random_state=42))
                           ])

scores = scores_statistics(estimator=gb_pipe_est, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

r2 mean: 0.6422642094394672
r2 std: 0.045081241398125756 

MAE mean: -122834.34681543028
MAE std: 4608.233441167535 

MAPE mean: -0.4030357828910572
MAPE std: 0.022817423528780835 

RMSE mean: -180430.7810183822
RMSE std: 8430.422906990498 



In [136]:
gboost = gb_pipe_est.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = gboost.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = gboost.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.6825632436915089
MAE: 116749.91257832848
MAPE: 0.38086430496936213
RMSE: 171416.84878404267

Performance on the test set:
 ----------
R²: 0.6400955083561792
MAE: 122317.9107731734
MAPE: 0.37215236374014693
RMSE: 184913.92579798016



## Extreme Gradient Boosting

In [20]:
from xgboost import XGBRegressor

In [21]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ])

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[levels_type_house, levels_code]))  #code_levels
                                      #handle_unknown='ignore'
    ])


preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'




xgb_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', XGBRegressor())
                           ])

In [27]:
grid_search = GridSearchCV(estimator=xgb_pipe_est, 
                           param_grid={'regressor__n_estimators': [75,80,85,90,95,100,105,110,115,120,125]},
                           cv=5, 
                           n_jobs=-1,
                        #   scoring='neg_mean_squared_error',
                        #   return_train_score=True
                          )

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'regressor__n_estimators': 105}
0.806309737415259


In [28]:
xgb_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', xgboost.XGBRegressor(n_estimators= 105))
                           ])

scores = scores_statistics(estimator=xgb_pipe_est, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

r2 mean: 0.8103256306854135
r2 std: 0.031107022249743918 

MAE mean: -76439.83677913898
MAE std: 3411.9054419745694 

MAPE mean: -0.21207778513758738
MAPE std: 0.007134753254566564 

RMSE mean: -131197.21047948365
RMSE std: 11002.37791559043 



In [29]:
xgb = xgb_pipe_est.fit(X_train, y_train)

print('Performance on the training set:\n', '-'*10)
y_pred_train = xgb.predict(X_train)
metrics_regression(y_test=y_train, 
                   y_pred=y_pred_train, 
                   squared=False)

print('Performance on the test set:\n', '-'*10)
y_pred = xgb.predict(X_test)
metrics_regression(y_test=y_test, 
                   y_pred=y_pred, 
                   squared=False)

Performance on the training set:
 ----------
R²: 0.9620408504154799
MAE: 40626.77943735192
MAPE: 0.13112311227814163
RMSE: 59276.57986186307

Performance on the test set:
 ----------
R²: 0.7992571055286443
MAE: 78311.53892506765
MAPE: 0.203086652634719
RMSE: 138100.76251426517

